In [10]:
import csv
import os
from pathlib import Path

import numpy as np

from compare_genes import getScores
from xmlparse import loadRef, loadGeometryBases, getXmlScore, minXml
from score_strokes import alignStrokes
from exhaustive import exhaustScore

## Scoring Functions

In [11]:
def heuristicScores(algo, ref_char, ref_dir, data_dir):
    heuristic_alignments = []
    heuristic_scores = []
    ref_geometry, ref_progress_percentage, output_size = loadRef(ref_char, ref_dir)
    g_data, _, base_data, stroke_sets, _, f_names = loadGeometryBases(data_dir, output_size)
    for (geometry_length, bases, stroke_set, _, f_name) in zip(g_data, base_data, stroke_sets, _, f_names):
        geometry, progress_percentage = geometry_length
        heuristic_alignment = np.array(algo(geometry, ref_geometry, progress_percentage, ref_progress_percentage))+1
        heuristic_alignments.append(heuristic_alignment)
        heuristic_xml = minXml(ref_char, bases, stroke_set, heuristic_alignment)
        heuristic_score = getXmlScore(heuristic_xml)
        heuristic_scores.append(heuristic_score)
    return heuristic_scores, heuristic_alignments

# Obtaining scores through exhaustive search
def exhaustiveScores(ref_char, data_dir, char_data):
    g_data, han_chars, base_data, _, _, f_names = char_data
    exhaustive_scores = []
    for (gl, han_char, bases, f_name) in zip(g_data, han_chars, base_data, f_names):
        g, l = gl
        original_score = exhaustScore(ref_char, f_name, data_dir, force_refresh=True, save=False)
        exhaustive_scores.append(original_score)
    return exhaustive_scores

## Gene/Archetype Combos

In [12]:
ref_dir = f'{str(Path.home())}/Stylus_Scoring_Generalization/Reference' # archetype directory
data_dir = f'{str(Path.home())}/Stylus_Scoring_Generalization/NewGenes' # gene directory

# Retrieve all reference characters
ref_chars = []
dirs = [f.path for f in os.scandir(ref_dir) if f.is_dir()]
for d in dirs:
    ref_chars.extend(f.split(".")[0] for f in os.listdir(d))
ref_chars = list(filter(None, ref_chars))

# Retrieve scores for every gene/archetype combo
with open('test.csv', 'w', newline='') as cf:
    writer = csv.writer(cf)
    writer.writerow(["GeneId", "ArchetypeId", "ExhaustiveScore", "HeuristicScore", "GeneMap", "HeuristicMap"])
gene_names = os.listdir(data_dir)
gene_names.sort()
for i, g in enumerate(gene_names):
    gene_names[i] = g.split(".gene")[0]
for ref_char in ref_chars:
    _, _, output_size = loadRef(ref_char, ref_dir)
    char_data = loadGeometryBases(data_dir, output_size)
    stroke_orders = char_data[4]
    heuristic_scores, heuristic_alignments = heuristicScores(alignStrokes, ref_char, ref_dir, data_dir)
    exhaustive_scores = exhaustiveScores(ref_char, data_dir, char_data)
    for (gene_name, heuristic_score, exhaustive_score, stroke_order, heuristic_alignment) in zip(gene_names, heuristic_scores, exhaustive_scores, stroke_orders, heuristic_alignments):
        writer.writerow([gene_name, ref_char, exhaustive_score, heuristic_score, stroke_order, heuristic_alignment])

KeyboardInterrupt: 

## Writing to CSV